In [1]:
!pip install git+https://gitlab.com/cortezpedro/chfem_gpu.git@dev
!pip install wurlitzer # to capture output from c function in colab
!pip install pyvista 

  Cloning https://gitlab.com/cortezpedro/chfem_gpu.git (to revision dev) to /tmp/pip-req-build-k6amz78d
  Running command git clone --filter=blob:none --quiet https://gitlab.com/cortezpedro/chfem_gpu.git /tmp/pip-req-build-k6amz78d
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  branch 'dev' set up to track 'origin/dev'.
  Resolved https://gitlab.com/cortezpedro/chfem_gpu.git to commit 0da686ba68d2313d1e26deaebda55aa923d54db3
  Preparing metadata (setup.py) ... done
^C
ERROR: Operation cancelled by user


In [2]:
from matplotlib import pyplot as plt
import pyvista as pv
import numpy as np
from wurlitzer import pipes
import chfem

ModuleNotFoundError: No module named 'wurlitzer'

In [ ]:
cube_size_a = 10
array_size_a = 20

def create_cube_array(array_side, cube_side, ntiles):
    cube = np.zeros((array_side, array_side, array_side), dtype=np.uint8)
    start_cube = array_size_a // 4
    end_cube = start_cube + cube_side
    cube[start_cube:end_cube, start_cube:end_cube, start_cube:end_cube] = 1
    cube_array = np.tile(cube, (ntiles, ntiles, ntiles))
    return cube_array

left = create_cube_array(20, 10, 10)
right = create_cube_array(40, 20, 5)
array = np.concatenate((left, right), axis=0)

In [ ]:
with pipes() as (out, err):
  keff = chfem.compute_conductivity(array, mat_props=[(255, 1), (0, 0.1)], direction='x', output_fields="cubes")
print(out.read(), err.read())

temperature = chfem.import_scalar_field_from_chfem("cubes_temperature_0.bin", array.shape, False)
plt.imshow(temperature[2])

In [ ]:
with pipes() as (out, err):
  Ceff = chfem.compute_elasticity(array, mat_props=[(255, 200, 0.2), (0, 100, 0.3)], direction='x', output_fields="cubes") 
print(out.read(), err.read())

displacement = chfem.import_vector_field_from_chfem("cubes_displacement_0.bin", array.shape, False)
plt.imshow(displacement[25, :, :, 0])

In [ ]:
with pipes() as (out, err):
  Keff = chfem.compute_permeability(array, direction='x', output_fields="cubes")
print(out.read(), err.read())

pressure = chfem.import_scalar_field_from_chfem("cubes_pressure_0.bin", array.shape, False)
plt.imshow(pressure[50])